In [7]:
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

In [8]:
client = QdrantClient(
    host="localhost",
    port=6333
)

In [37]:
client.get_collections(
    
)

CollectionsResponse(collections=[CollectionDescription(name='norma_ai_collection_ru-en-RoSBERTa')])

In [9]:
model_name = "ai-forever/ru-en-RoSBERTa"
model_kwargs = {"device":"cpu"}
encode_kwargs = {'normalize_embeddings': True, 'batch_size': 128}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [10]:
vs = QdrantVectorStore(
    client=client, 
    collection_name="norma_ai_collection_ru-en-RoSBERTa",
    embedding=embeddings_model,
)

In [11]:
query = "На какой максимальной высоте разрешается устанавливать штурвал ЗРА без применения площадок обслуживания"
results_with_scores = vs.similarity_search_with_score(
    query,
    k=7
)

for i, (doc, score) in enumerate(results_with_scores):
    print(f"\n--- Результат {i+1} ---")
    print(f"Описание: {doc.page_content[:]}")


--- Результат 1 ---
Описание: быть не менее 1 м. Люлька должна быть застрахована от падения.
350. При механизированном осуществлении спускоподъемных операций без участия верхового
КонсультантПлюс
www.consultant.ru Страница 37 из 312
надежная правовая поддержка

--- Результат 2 ---
Описание: "Руководство по безопасности "Рекомендации по
Документ предоставленКонсультантПлюс
техническому диагностированию сварных вертикальных
Дата сохранения: 09.01.2024
цилиндрических резервуа...
осадки основания) рекомендуется ежегодно проводить нивелирование окрайки днища в абсолютных
отметках и результаты заносить в журнал нивелирования окрайки днища.
26. Нивелирование стационарных стальных и алюминиевых крыш проводится для определения
отклонений высотных отметок несущих конструкций стальных и алюминиевых стационарных крыш от
значений, приведенных в проектной документации, в случае выявления видимых деформаций по
результатам ВИК.
27. Нивелирование стальных крыш проводится с площадок на крыше резервуара

## LLM

In [12]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from dotenv import dotenv_values

In [14]:
groq_api_key = dict(dotenv_values())['GROQ_API_KEY']

In [15]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

os.environ["GROQ_API_KEY"] = groq_api_key

In [16]:
import os
print("GROQ_API_KEY set:", bool(os.getenv("GROQ_API_KEY")))
print("ALL_PROXY:", os.getenv("ALL_PROXY"))
print("HTTPS_PROXY:", os.getenv("HTTPS_PROXY"))


GROQ_API_KEY set: True
ALL_PROXY: None
HTTPS_PROXY: None


In [17]:
import os, httpx
from langchain_groq import ChatGroq

# убеждаемся, что ключ и прокси заданы до создания клиента
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
proxy = "socks5h://127.0.0.1:10808"

http_client = httpx.Client(proxy=proxy, timeout=60)

In [51]:
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=2000,
    http_client=http_client,
)

In [32]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import getpass

In [60]:
question = 'Что такое внутритрубная диагностика?'

In [44]:
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты — эксперт по нормативно-технической документации нефтегазовой отрасли.  
Тебе дан вопрос пользователя и фрагменты документов (результаты поиска в векторной базе).  

Твоя задача:  
- Суммаризировать и структурировать найденные фрагменты.  
- Построить ответ на вопрос пользователя, основываясь только на данных из этих фрагментов.  
- Не придумывать информации, которая отсутствует.  
- В ответ включать конкретные детали, ссылки на пункты/статьи/разделы, если они явно указаны во фрагментах.  
- Если фрагменты содержат противоречивые сведения, укажи на это.  
- Если информации недостаточно, явно напиши: «В предоставленных фрагментах отсутствует информация для полного ответа».  

Формат ответа:  
1. Краткое резюме сути.  
2. Детали и пояснения (с конкретными формулировками из документов).  
3. Итоговое заключение.

---

Вопрос пользователя:
{question}"""),
    ("human", """Вот документы, нашедшиеся по твоему запросу:\n\n{context}
    \n\nПожалуйста, сделай обзор и дай дополнительные рекомендации!""")
])

In [61]:
# допустим, есть список документов от векторного поиска
retrieved_docs = vs.similarity_search(question, k=5)

# превращаем их в строку
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# формируем промпт
formatted_prompt = rag_prompt.format(question=question, context=context)

print(formatted_prompt)

System: Ты — эксперт по нормативно-технической документации нефтегазовой отрасли.  
Тебе дан вопрос пользователя и фрагменты документов (результаты поиска в векторной базе).  

Твоя задача:  
- Суммаризировать и структурировать найденные фрагменты.  
- Построить ответ на вопрос пользователя, основываясь только на данных из этих фрагментов.  
- Не придумывать информации, которая отсутствует.  
- В ответ включать конкретные детали, ссылки на пункты/статьи/разделы, если они явно указаны во фрагментах.  
- Если фрагменты содержат противоречивые сведения, укажи на это.  
- Если информации недостаточно, явно напиши: «В предоставленных фрагментах отсутствует информация для полного ответа».  

Формат ответа:  
1. Краткое резюме сути.  
2. Детали и пояснения (с конкретными формулировками из документов).  
3. Итоговое заключение.

---

Вопрос пользователя:
Что такое внутритрубная диагностика?
Human: Вот документы, нашедшиеся по твоему запросу:

эксплуатацию, рекомендуется производить после его в

In [46]:
context

'быть не менее 1 м. Люлька должна быть застрахована от падения.\n350. При механизированном осуществлении спускоподъемных операций без участия верхового\nКонсультантПлюс\nwww.consultant.ru Страница 37 из 312\nнадежная правовая поддержка\n\n39. Объем УЗК контроля сварных соединений стенки резервуара при полном техническом\nдиагностировании приведен в таблице N 2 приложения N 6 к настоящему Руководству по безопасности\nили может быть заменен на РК в соответствии с ГОСТ 7512-82 "Контроль неразрушающий. Соединения\nсварные. Радиографический метод".\n40. Перекрестья сварных соединений резервуара для УЗК рекомендуется выбирать по результатам\nпредварительно проведенного анализа документации и ВИК.\n41. При контроле сварных соединений резервуара, приведенных в таблице N 2 приложения N 6 к\nнастоящему Руководству по безопасности, в случае обнаружения недопустимых дефектов объем\nконтроля (длина сварных швов и количество контролируемых перекрестий) удваивается. В случае\nповторного обнаружения н

In [80]:
scroll_result, _ = client.scroll(
    collection_name='norma_ai_collection_ru-en-RoSBERTa',
    offset=124,
    limit=1,
    with_payload=True,
    with_vectors=True,
)

point = scroll_result[0]

In [41]:
point.payload['metadata']


{'source': '/srv/norma_ai/norma_ai_project/data/Федеральный закон от 31.07.2020 N 248-ФЗ О государственном контроле (надзоре) и муниципальном.pdf',
 'file_path': '/srv/norma_ai/norma_ai_project/data/Федеральный закон от 31.07.2020 N 248-ФЗ О государственном контроле (надзоре) и муниципальном.pdf',
 'page': 45,
 'total_pages': 98,
 'Author': 'Evgeniy Ostromecky',
 'Creator': 'Writer',
 'Producer': 'LibreOffice 24.2',
 'CreationDate': "D:20250827112736Z'"}

In [85]:
format_docs(point)

AttributeError: 'tuple' object has no attribute 'metadata'

In [65]:
def format_docs(docs):
    """Форматирует документы для передачи в промпт"""
    formatted = []

    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata

        vacancy_info = f"""
        === Документ {i} ===
        Название: {metadata.get('source', 'Не указано')}
        Страница: {metadata.get('page', 'Не указано')}
        Всего страниц: {metadata.get('total_pages', 'Не указано')}
        Дата создания: {metadata.get('CreationDate', 'Не указано')}

        Описание: {doc.page_content[:300]}...
        """

        formatted.append(vacancy_info)

    return "\n".join(formatted)

print("✅ Функция форматирования создана")

✅ Функция форматирования создана


In [66]:
retriever = vs.as_retriever(
    search_type='similarity',
    search_kwargs={"k": 5}
)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [67]:
query = "Что такое внутритрубная диагностика?"
answer = rag_chain.invoke(query)
print("🔍 Рекомендации:")
print(answer)

🔍 Рекомендации:
<think>
Хорошо, я получил вопрос пользователя: "Что такое внутритрубная диагностика?" и пять фрагментов документов. Мне нужно суммировать и структурировать эти фрагменты, чтобы построить ответ.

Сначала я посмотрю на каждый документ:

Документ 1 (страница 3) говорит о том, что эксплуатацию рекомендуется проводить после ввода в эксплуатацию в течение года. Также упоминается, что типы внутритрубных приборов определяет заказчик.

Документ 2 (страница 16) рассказывает о видах наружного диагностирования для спиральношовных труб, включая ВТД для выявления дефектов в швах.

Документ 3 (страница 38) определяет внутритрубное диагностирование как вид технического диагностирования, связанного с использованием оптических приборов и средств измерения.

Документ 4 (страница 39) упоминает приказ Ростехнадзора №330 и связывает его с безопасностью, техническим диагностированием и параметрами дефектов после ВТД.

Документ 5 (страница 26) рекомендует измерять толщину стенок в четырех точк

In [ ]:
resp = llm.invoke("Привет, как твои дела?")


In [ ]:
resp.content

NameError: name 'resp' is not defined

In [ ]:
# import os
# import getpass
# from langchain_mistralai.chat_models import ChatMistralAI

# # Безопасный ввод ключа (не отображается в консоли)
# os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter Mistral API Key: ")

# llm_2 = ChatMistralAI(
#     model="mistral-medium",    # или нужную тебе модель из документации Mistral
#     temperature=0,
#     max_tokens=1024
# )

In [ ]:
# resp_2 = llm.invoke("Привет, как твои дела?")